In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from statistics import median
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import class_weight
import warnings


In [51]:
df = pd.read_csv("renamed_reco_train.csv")

date_columns = ['report_date', 'contract_start_date']  # Add any other date columns if necessary
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')
# Calculate the difference in days
df['contract_length'] = (df['report_date'] - df['contract_start_date']).dt.days

# Insert 'contract_length' in the same spot as 'contract_start_date'
start_date_index = df.columns.get_loc('contract_start_date')
df.insert(start_date_index, 'contract_length', df.pop('contract_length'))

# Drop the original 'contract_start_date' and 'report_date' columns
df = df.drop(['contract_start_date', 'report_date', 'customer_id'], axis='columns')
#replace missing values in gross income and age with median of distribution
count = df['gross_income'].isna().sum()
df['gross_income'] = df['gross_income'].fillna(df['gross_income'].median())
df.head(10)
age_new = df.iloc[:,3]
print(len(age_new))
age_new[1020:1040]
new = []
for i in age_new:
    if i != ' NA':
        new.append(int(i))

med = median(new)
med
age = []
for i in age_new:
    if i !=' NA':
        age.append(int(i))
    else:
        age.append(med)
age[1020:1040]
df['age'] = age

df['country_residence'].unique()
#has NA and many countries one hot encode
count = df['country_residence'].isna().sum()
#dropping row if column country_residenceis NA
df = df.dropna(subset=["country_residence"])
len(df)
df['gender'].unique()  
# binary one hot encode
df['new_customer_index'].unique() 
# binary one hot encode
df['seniority_months'].unique() 
#conversion from string to int
new = []
for i in df['seniority_months']:
    new.append(int(i))
df['seniority_months'] = new

df['customer_relation_type'].unique()
# binary one hot encode
df['residence_index'].unique()
# binary one hot encode
df['foreigner_index'].unique()
# binary one hot encode
df['join_channel'].unique()
#one hot encoding for many variables
df['deceased_index'].unique()
# binary one hot encode
df['province_name'].unique()
#one hot encoding for many variables
df['customer_segment'].unique()
#one hot encoding for many variables
df['employee_index'].unique()
df = df.drop('employee_index', axis='columns')
df = pd.get_dummies(df, columns=['customer_segment','province_name','join_channel','country_residence'], dtype = int)
#one hot encoding for many variables
df = pd.get_dummies(df, columns=['deceased_index','foreigner_index','residence_index','customer_relation_type','gender','new_customer_index' ], drop_first=True, dtype = int)
# binary one hot encode
df
cols = df.columns.tolist()
cols
cols = cols[:5]+cols[9:] + cols[5:9]
df = df[cols]
# Move 'account' column to the last position
account_column = df.pop('account')  # Remove the column and save it
df['account'] = account_column
df

9999


,age,contract_length,seniority_months,primary_customer_status,customer_type_start_month,customer_segment_01 - TOP,customer_segment_02 - PARTICULARES,customer_segment_03 - UNIVERSITARIO,province_name_ALAVA,province_name_ALBACETE,...,foreigner_index_S,residence_index_S,customer_relation_type_I,gender_V,new_customer_index_1.0,gross_income,fixed_deposits,loan,credit_card_debit_card,account
0,35,16.0,6,1.0,1.0,0,1,0,0,0,...,0,1,0,0,0,87218.100,0,0,0,1
1,23,901.0,35,1.0,1.0,0,0,1,0,0,...,1,1,1,1,0,35548.740,0,0,0,1
2,23,901.0,35,1.0,1.0,0,0,1,0,0,...,0,1,1,1,0,122179.110,0,0,0,1
3,22,901.0,35,1.0,1.0,0,0,1,0,0,...,0,1,1,0,0,119775.540,1,0,0,0
4,23,901.0,35,1.0,1.0,0,0,1,0,0,...,0,1,0,1,0,87835.005,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,23,856.0,34,1.0,1.0,0,0,1,0,0,...,0,1,0,0,0,156031.680,0,0,0,1
9995,27,856.0,34,1.0,1.0,0,0,1,0,0,...,0,1,1,1,0,79357.410,0,0,0,1
9996,24,856.0,34,1.0,1.0,0,0,1,0,0,...,0,1,1,1,0,652313.070,0,0,0,1
9997,23,856.0,34,1.0,1.0,0,0,1,0,0,...,0,1,1,0,0,130379.970,0,0,0,1


### Faster but less thorough version.

In [52]:
import warnings
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Ignore ConvergenceWarning
warnings.filterwarnings("ignore", category=UserWarning, message=".*ConvergenceWarning.*")

# Separate features and target variables
prediction_columns = ['fixed_deposits', 'loan', 'credit_card_debit_card', 'account']
X = df.drop(columns=prediction_columns)  # First 86 columns (features)
y = df[prediction_columns]

# Normalize the feature columns
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Create a class for model evaluation
class ModelEvaluator:
    def __init__(self):
        self.models = {}
        
    def train_models(self, X_train, y_train):
        # Define the logistic regression model
        lr_model = LogisticRegression(max_iter=1000, class_weight='balanced', solver='liblinear')

        # Set up hyperparameters for tuning
        param_dist = {
            'C': [0.001, 0.01, 0.1, 1],  # Reduced number of hyperparameters
            'penalty': ['l1', 'l2'],  # Only two regularization types
        }

        # Use StratifiedKFold for cross-validation
        skf = StratifiedKFold(n_splits=3)  # Reduced number of folds

        # Perform random search with cross-validation
        random_search = RandomizedSearchCV(lr_model, param_dist, n_iter=10, cv=skf, scoring='f1_macro', n_jobs=-1)

        for i in range(y_train.shape[1]):
            print(f"Training model for target column {y_train.columns[i]}")
            # Train the model
            random_search.fit(X_train, y_train.iloc[:, i])
            self.models[y_train.columns[i]] = random_search.best_estimator_
            print(f"Best parameters for {y_train.columns[i]}: {random_search.best_params_}")

    def evaluate_model(self, model, X_test, y_test, column_name):
        y_pred = model.predict(X_test)
        print(f"Confusion Matrix for {column_name}:\n", confusion_matrix(y_test[column_name], y_pred))
        print(f"Classification Report for {column_name}:\n", classification_report(y_test[column_name], y_pred))
        
    def evaluate_all_models(self, X_test, y_test):
        for column_name, model in self.models.items():
            self.evaluate_model(model, X_test, y_test, column_name)

# Create and train the models
model_evaluator = ModelEvaluator()
model_evaluator.train_models(X_train, y_train)

# Evaluate all trained models
model_evaluator.evaluate_all_models(X_test, y_test)

# After training, you can use the models to make predictions on new data
def predict_proba(new_data):
    new_data_normalized = scaler.transform(new_data)
    probabilities = {}
    for column_name, model in model_evaluator.models.items():
        probabilities[column_name] = model.predict_proba(new_data_normalized)[:, 1]  # Probability of class 1
    return probabilities


Training model for target column fixed_deposits


/Users/collinsng/anaconda3/envs/dsa3101/lib/python3.9/site-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best parameters for fixed_deposits: {'penalty': 'l1', 'C': 0.1}
Training model for target column loan


/Users/collinsng/anaconda3/envs/dsa3101/lib/python3.9/site-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best parameters for loan: {'penalty': 'l1', 'C': 1}
Training model for target column credit_card_debit_card


/Users/collinsng/anaconda3/envs/dsa3101/lib/python3.9/site-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best parameters for credit_card_debit_card: {'penalty': 'l2', 'C': 1}
Training model for target column account


/Users/collinsng/anaconda3/envs/dsa3101/lib/python3.9/site-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best parameters for account: {'penalty': 'l1', 'C': 0.001}
Confusion Matrix for fixed_deposits:
 [[1821  155]
 [   3   11]]
Classification Report for fixed_deposits:
               precision    recall  f1-score   support

           0       1.00      0.92      0.96      1976
           1       0.07      0.79      0.12        14

    accuracy                           0.92      1990
   macro avg       0.53      0.85      0.54      1990
weighted avg       0.99      0.92      0.95      1990

Confusion Matrix for loan:
 [[1930   58]
 [   2    0]]
Classification Report for loan:
               precision    recall  f1-score   support

           0       1.00      0.97      0.98      1988
           1       0.00      0.00      0.00         2

    accuracy                           0.97      1990
   macro avg       0.50      0.49      0.49      1990
weighted avg       1.00      0.97      0.98      1990

Confusion Matrix for credit_card_debit_card:
 [[1377  447]
 [  21  145]]
Classification Rep

### This model takes a while to run, its the more thorough version. Select the code and command + / for mac to undo the # . 

In [53]:
# # Separate features and target variables
# prediction_columns = ['fixed_deposits', 'loan', 'credit_card_debit_card', 'account']
# X = df.drop(columns=prediction_columns)  # First 86 columns (features)
# y = df[prediction_columns]

# # Normalize the feature columns
# scaler = StandardScaler()
# X_normalized = scaler.fit_transform(X)

# # Should use, stratified train-test split to maintain class distribution, but errors encountered ()
# # one of the classes is has too few rows
# X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# # Create a class for model evaluation
# class ModelEvaluator:
#     def __init__(self):
#         self.models = {}
        
#     def train_models(self, X_train, y_train):
#         # Define the logistic regression model
#         lr_model = LogisticRegression(max_iter=1000, class_weight='balanced')

#         # Set up hyperparameters for tuning
#         param_grid = {
#             'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Inverse of regularization strength
#             'penalty': ['l1', 'l2', 'elasticnet', 'none'],  # Regularization type
#             'solver': ['liblinear', 'saga']  # Solver to use
#         }

#         # Use StratifiedKFold for cross-validation
#         skf = StratifiedKFold(n_splits=5)

#         # Perform grid search with cross-validation
#         grid_search = GridSearchCV(lr_model, param_grid, cv=skf, scoring='f1_macro', n_jobs=-1)
        
#         for i in range(y_train.shape[1]):
#             print(f"Training model for target column {y_train.columns[i]}")
#             # Train the model
#             grid_search.fit(X_train, y_train.iloc[:, i])
#             self.models[y_train.columns[i]] = grid_search.best_estimator_
#             print(f"Best parameters for {y_train.columns[i]}: {grid_search.best_params_}")



#     def evaluate_model(self, model, X_test, y_test, column_name):
#         y_pred = model.predict(X_test)
#         print(f"Confusion Matrix for {column_name}:\n", confusion_matrix(y_test[column_name], y_pred))
#         print(f"Classification Report for {column_name}:\n", classification_report(y_test[column_name], y_pred))
        
#     def evaluate_all_models(self, X_test, y_test):
#         for column_name, model in self.models.items():
#             self.evaluate_model(model, X_test, y_test, column_name)

# # Create and train the models
# model_evaluator = ModelEvaluator()
# model_evaluator.train_models(X_train, y_train)

# # Evaluate all trained models
# model_evaluator.evaluate_all_models(X_test, y_test)

# # After training, you can use the models to make predictions on new data
# def predict_proba(new_data):
#     new_data_normalized = scaler.transform(new_data)
#     probabilities = {}
#     for column_name, model in model_evaluator.models.items():
#         probabilities[column_name] = model.predict_proba(new_data_normalized)[:, 1]  # Probability of class 1
#     return probabilities




Training model for target column fixed_deposits


/Users/collinsng/anaconda3/envs/dsa3101/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/collinsng/anaconda3/envs/dsa3101/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/collinsng/anaconda3/envs/dsa3101/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/collinsng/anaconda3/envs/dsa3101/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/collinsng/anaconda3/envs/dsa3101/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  

KeyboardInterrupt: 